## VN30F1M - 5min timeframe - RSI analytics

In [2]:
import warnings
warnings.filterwarnings('ignore')

import time
from datetime import date, datetime
from datetime import timedelta
from dateutil.relativedelta import relativedelta

import matplotlib.pyplot as plt
import pandas as pd
import pandas_ta as ta
import numpy as np

In [3]:
import os
from pathlib import Path
notebook_path = os.getcwd()
algo_dir = Path(notebook_path).parent.parent
csv_file = str(algo_dir) + '/vn-stock-data/VN30ps/VN30F1M_5minutes.csv'
is_file = os.path.isfile(csv_file)
if is_file:
    dataset = pd.read_csv(csv_file, index_col='Date', parse_dates=True)
else:
    dataset = pd.read_csv("https://raw.githubusercontent.com/zuongthaotn/vn-stock-data/main/VN30ps/VN30F1M_5minutes.csv", index_col='Date', parse_dates=True)

In [4]:
data = dataset.copy()
data["RSI_10"] = ta.rsi(data["Close"], length=10)
data["RSI_10_shift"] = data["RSI_10"].shift(1)
data.dropna(inplace=True)
data

,Open,High,Low,Close,Volume,RSI_10,RSI_10_shift
Date,,,,,,,
2018-08-13 09:55:00,946.2,946.4,945.6,946.0,1873,72.619617,72.619617
2018-08-13 10:00:00,945.9,946.3,945.5,945.5,1704,64.781384,72.619617
2018-08-13 10:05:00,945.5,946.0,945.0,945.0,1436,57.844224,64.781384
2018-08-13 10:10:00,945.0,945.3,944.5,945.2,1196,59.759420,57.844224
2018-08-13 10:15:00,945.0,945.7,944.9,945.2,1401,59.759420,59.759420
...,...,...,...,...,...,...,...
2024-06-19 14:15:00,1320.6,1325.0,1319.0,1322.7,15368,88.598172,87.013227
2024-06-19 14:20:00,1322.6,1323.0,1317.0,1317.0,12067,66.312992,88.598172
2024-06-19 14:25:00,1317.3,1318.0,1314.3,1314.5,11350,59.072035,66.312992


In [5]:
# Ignore all candlesticks at 14:30 & 14:45
data = data[(data.index.hour != 14) | ((data.index.hour == 14) & (data.index.minute < 30))]

In [18]:
trading_data = data[(100 * data.index.hour + data.index.minute > 915) & (100 * data.index.hour + data.index.minute < 1435)]

,Open,High,Low,Close,Volume,RSI_10,RSI_10_shift
Date,,,,,,,
2018-08-13 09:55:00,946.2,946.4,945.6,946.0,1873,72.619617,72.619617
2018-08-13 10:00:00,945.9,946.3,945.5,945.5,1704,64.781384,72.619617
2018-08-13 10:05:00,945.5,946.0,945.0,945.0,1436,57.844224,64.781384
2018-08-13 10:10:00,945.0,945.3,944.5,945.2,1196,59.759420,57.844224
2018-08-13 10:15:00,945.0,945.7,944.9,945.2,1401,59.759420,59.759420
...,...,...,...,...,...,...,...
2024-06-19 14:05:00,1309.6,1316.7,1309.6,1316.7,14553,83.740938,71.214716
2024-06-19 14:10:00,1316.7,1321.3,1314.9,1320.4,17070,87.013227,83.740938
2024-06-19 14:15:00,1320.6,1325.0,1319.0,1322.7,15368,88.598172,87.013227


# Calculating return

In [26]:
trading_data['return'] = ''
for i, row in trading_data.iterrows():
    if row['RSI_10_shift'] < 30 and row['RSI_10'] >= 30 and (100 * row.name.hour + row.name.minute) < 1430:
        # Long signal
        current_date = row.name.strftime('%Y-%m-%d ').format()
        current_time = row.name
        entry_price = row['Close']
        data_to_end_day = trading_data[(trading_data.index > current_time) & (trading_data.index < current_date+' 14:30:00')]
        max_price = 0
        for k, wrow in data_to_end_day.iterrows():
            if wrow['Close'] < entry_price and wrow['Close'] < entry_price - 3.5:
                momentum = -3.5
                break
            else:
                if max_price < wrow['High']:
                    max_price = wrow['High']
                if wrow['Close'] < max_price - 4:
                    momentum = wrow['Close'] - entry_price
                    break
                else:
                    momentum = wrow['Close'] - entry_price
        trading_data.at[i, 'return'] = momentum
# trading_data

## RSI cross up 30 analytics

In [27]:
rsi_cross_up_data = trading_data[(trading_data.RSI_10_shift < 30) & (trading_data.RSI_10 >= 30)]
len(rsi_cross_up_data)

1793

In [28]:
rsi_cross_up_data['diff'] = rsi_cross_up_data['RSI_10'] - rsi_cross_up_data['RSI_10_shift']
rsi_cross_up_data

,Open,High,Low,Close,Volume,RSI_10,RSI_10_shift,return,diff
Date,,,,,,,,,
2018-08-15 11:25:00,956.5,958.2,956.5,958.2,1603,44.805183,29.490656,-3.5,15.314528
2018-08-16 09:55:00,940.4,941.6,940.1,941.5,951,30.419285,22.875730,-2.6,7.543556
2018-08-16 11:15:00,936.3,937.1,935.5,937.1,1260,30.891782,22.480256,7.4,8.411527
2018-08-17 14:00:00,947.1,947.4,946.4,947.4,2631,32.448772,25.260240,-0.4,7.188532
2018-08-20 14:25:00,944.5,945.4,944.3,944.7,3895,32.757259,29.889019,-0.4,2.868240
...,...,...,...,...,...,...,...,...,...
2024-06-11 11:05:00,1305.4,1306.1,1305.0,1305.9,2212,34.423450,26.612215,-3.3,7.811234
2024-06-11 13:35:00,1300.7,1303.0,1300.3,1302.6,6103,34.293371,15.738483,5.6,18.554889
2024-06-17 10:10:00,1310.4,1312.1,1310.0,1311.8,3197,35.254427,28.445897,-1.8,6.808530


In [31]:
rsi_cross_up_positive_data = rsi_cross_up_data[rsi_cross_up_data['return'] > 0]
rsi_cross_up_positive_data['diff'].describe()

count    679.000000
mean       9.768161
std        5.638874
min        0.725493
25%        5.647145
50%        8.818991
75%       12.830067
max       32.413474
Name: diff, dtype: float64

In [33]:
rsi_cross_up_positive_data['return'].sum()

3275.2

In [32]:
rsi_cross_up_negative_data = rsi_cross_up_data[rsi_cross_up_data['return'] < 0]
rsi_cross_up_negative_data['diff'].describe()

count    1097.000000
mean        9.081550
std         5.252047
min         0.294189
25%         5.116422
50%         8.242791
75%        12.093785
max        35.304006
Name: diff, dtype: float64

In [34]:
rsi_cross_up_negative_data['return'].sum()

-2946.4000000000015

In [11]:
selected_date_data = data[(data.index > '2024-06-19 00:00:00') & (data.index < '2024-06-19 15:00:00')]
selected_date_data

,Open,High,Low,Close,Volume,RSI_10,RSI_10_shift
Date,,,,,,,
2024-06-19 09:00:00,1309.8,1311.0,1309.8,1310.9,5161,37.253890,24.882356
2024-06-19 09:05:00,1311.1,1311.9,1310.9,1311.7,2895,40.861254,37.253890
2024-06-19 09:10:00,1311.7,1311.8,1311.2,1311.6,2166,40.537565,40.861254
2024-06-19 09:15:00,1311.5,1311.9,1310.0,1311.1,3993,38.828741,40.537565
2024-06-19 09:20:00,1311.1,1311.8,1310.7,1311.5,1642,41.038063,38.828741
2024-06-19 09:25:00,1311.5,1311.6,1310.3,1310.6,2173,37.639499,41.038063
2024-06-19 09:30:00,1310.6,1310.6,1308.5,1309.0,4184,32.347872,37.639499
2024-06-19 09:35:00,1308.9,1310.5,1308.9,1310.0,3294,38.365266,32.347872
2024-06-19 09:40:00,1310.1,1310.3,1309.2,1309.3,2588,35.882877,38.365266
